In [55]:
import pandas as pd
import numpy as np



In [56]:
file_name = "part-r-00000"
columns_list = ['主力净流入','主力流入','主力流出','超大买单成交量','超大买单成交额','超大卖单成交量','超大卖单成交额','大买单成交量','大买单成交额','大卖单成交量','大卖单成交额','中买单成交量','中买单成交额','中卖单成交量','中卖单成交额','小买单成交量','小买单成交额','小卖单成交量','小卖单成交额','时间范围']

time_list = []

price_table = pd.DataFrame(columns=columns_list)
with open(file_name) as f:
    while True:
        line = f.readline()
        if not line:
            break
        separate_by_space = line.split('\t')
        separate_by_colon = separate_by_space[1].split(',')
        separate_by_slash = separate_by_space[0].split('_')
        order_type = separate_by_slash[0]
        order_id = separate_by_slash[1]
        order_time = separate_by_slash[2]
        order_size = separate_by_slash[3]
        order_qty = float(separate_by_colon[0])
        order_price = float(separate_by_colon[1].strip())
        time_list.append(order_time)

        if order_time not in price_table['时间范围'].values:
            new_row = pd.DataFrame({'时间范围': [order_time]})
            price_table = pd.concat([price_table, new_row], ignore_index=True)

        if order_type == 'Buy':
            if order_size == 'ExtraLarge':
                price_table.loc[price_table['时间范围'] == order_time, '超大买单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '超大买单成交额'] = order_price
            elif order_size == 'Large':
                price_table.loc[price_table['时间范围'] == order_time, '大买单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '大买单成交额'] = order_price
            elif order_size == 'Medium':
                price_table.loc[price_table['时间范围'] == order_time, '中买单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '中买单成交额'] = order_price
            elif order_size == 'Small':
                price_table.loc[price_table['时间范围'] == order_time, '小买单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '小买单成交额'] = order_price
        elif order_type == 'Sell':
            if order_size == 'ExtraLarge':
                price_table.loc[price_table['时间范围'] == order_time, '超大卖单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '超大卖单成交额'] = order_price
            elif order_size == 'Large':
                price_table.loc[price_table['时间范围'] == order_time, '大卖单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '大卖单成交额'] = order_price
            elif order_size == 'Medium':
                price_table.loc[price_table['时间范围'] == order_time, '中卖单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '中卖单成交额'] = order_price
            elif order_size == 'Small':
                price_table.loc[price_table['时间范围'] == order_time, '小卖单成交量'] = order_qty
                price_table.loc[price_table['时间范围'] == order_time, '小卖单成交额'] = order_price






In [57]:
columns_list_2 = columns_list.copy()
columns_list_2.remove('时间范围')
price_table.fillna(0, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9892\3023856724.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_table.fillna(0, inplace=True)


In [58]:
type(price_table['超大卖单成交量'][0])

numpy.float64

In [60]:

for time in time_list:
    price_table.loc[price_table['时间范围'] == time, '主力流入'] = price_table.loc[price_table['时间范围'] == time, '超大买单成交额'] + price_table.loc[price_table['时间范围'] == time, '大买单成交额']
    price_table.loc[price_table['时间范围'] == time, '主力流出'] = price_table.loc[price_table['时间范围'] == time, '超大卖单成交额'] + price_table.loc[price_table['时间范围'] == time, '大卖单成交额']
    price_table.loc[price_table['时间范围'] == time, '主力净流入'] = price_table.loc[price_table['时间范围'] == time, '主力流入'] - price_table.loc[price_table['时间范围'] == time, '主力流出']

    


price_table.sort_values(by ='时间范围', ascending=True, inplace=True)
price_table.reset_index(drop=True, inplace=True)
price_table.to_csv('price_table_sorted.csv', index=False,encoding='gbk')